# NRPy+'s Finite Difference Interface

### NRPy+ Source Code for this module: [finite_difference.py](../edit/finite_difference.py)

### Finite difference derivatives

Suppose we have a *uniform* numerical grid in one dimension; say, the Cartesian $x$ direction. Since the grid is uniform, the spacing between successive grid points is $\Delta x$, and the position of the $i$th point is given by

$$x_i = x_0 + i \Delta x.$$

Then, given a function $u(x)$ on this uniform grid, we will adopt the notation

$$u(x_i) = u_i.$$

We wish to approximate derivatives of $u_i$ at some nearby point (first we will consider derivatives at $x_i$ itself) using [finite difference](https://en.wikipedia.org/wiki/Finite_difference). (FD) techniques. 

FD techniques are usually equivalent to first finding the unique $N$th degree polynomial that passes through $N+1$ sampled points of our function ($u$) in the neighborhood of where we wish to find the derivative. Then, provided $u$ is smooth and properly-sampled, the $M$th derivative of the polynomial (where $M\le N-1$; *Exercise: Justify this inequality*) is an approximate $M$th derivative of $u$ called the *$M$th-order finite difference derivative*. The approximation error generally decreases as the polynomial degree or sampling density increases.

Finite difference derivatives are written in the form
$$u^{(n)}(x_i)_{\text{FD}} = \sum_{j} u_j a_j,$$
where the $a_j$'s are known as *finite difference coefficients*. For a given finite difference representation, the set of $a_j$'s are unique.

There are many ways to compute finite difference coefficients $a_j$, though perhaps the most popular involves Taylor series expansions about sampled points near the point where we wish to evaluate the derivative.

### Recommended: Learn more about the algorithm NRPy+ adopts to automatically compute finite difference derivatives: ([How NRPy+ Computes Finite Difference Coefficients](Tutorial-How_NRPy_Computes_Finite_Difference_Coeffs.ipynb))

The finite_difference NRPy+ module contains one parameter:
* **FD_CENTDERIVS_ORDER**: An integer indicating the requested finite difference *accuracy* order (*not* the order of the derivative) , where FD_CENTDERIVS_ORDER = \[the size of the finite difference stencil in each direction, plus one\]. 

The finite_difference NRPy+ module contains two core functions: **compute_fdcoeffs_fdstencl** and **FD_outputC()**. The first is a low-level function normally called only by FD_outputC(), which computes and outputs finite difference coefficients and the numerical grid indices (stencil) corresponding to each coefficient:

### **compute_fdcoeffs_fdstencl(derivstring,FDORDER=-1)**:
* Output nonzero finite difference coefficients and corresponding numerical stencil as lists, using as inputs:
    * **derivstring**: a string of $N$ consecutive "D"'s, where $N$ corresponds to the order of the derivative, followed by a set of $N$ integers that denote the direction of each derivative. For example, in 3D Cartesian coordinates (x,y,z),
        * the derivative operator $\partial_x^2$ corresponds to derivstring = "DD00"
        * the derivative operator $\partial_x \partial_y$ corresponds to derivstring = "DD01"
        * the derivative operator $\partial_z$ corresponds to derivstring = "D2"
    * **FDORDER**: an *optional* parameter that, if set to a positive even integer, overrides FD_CENTDERIVS_ORDER

Normally compute_fdcoeffs_fdstencl() is only called from the other function within NRPy+'s finite_difference module, FD_outputC(). Regardless, this function is so simple that it makes

In [3]:
# Import the finite difference module
import finite_difference as fin

# 
fdcoeffs, fdstencl = fin.compute_fdcoeffs_fdstencl("D0",4)
print(fdcoeffs)
print(fdstencl)

[1/12, -2/3, 2/3, -1/12]
[[-2, 0, 0, 0], [-1, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0]]


Interpreting the output, notice first that $\texttt{fdstencl}$ is a list of coordinate indices, where up to 4 dimension indices are supported (higher dimensions are possible and can be straightforwardly added, though be warned about [The Curse of Dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality)).

Thus NRPy+ found that for some function $u$, the fourth-order accurate finite difference operator is given by

$$[\partial_x u]_\text{FD4} = \frac{1}{12} \left(u_{i0-2,i1,i2,i3} - u_{i0+2,i1,i2,i3}\right) + \frac{2}{3} \left(-u_{i0-1,i1,i2,i3} + u_{i0+1,i1,i2,i3}\right)$$

If $u$ is a function of fewer than four coordinate variables, it is up to the calling function to truncate the additional information.

### **FD_outputC(filename,sympyexpr_list)**:

FD_outputC() is the 

In [2]:
import sympy as sp
from outputC import *
import grid as gri
import finite_difference as fin

phi, phiD0 = gri.register_gridfunctions("AUX",["phi","phiD0"])

phi_dD0 = sp.sympify("phi_dD0")
fin.FD_outputC("stdout",lhrh(lhs=gri.gfaccess("out_gfs","phiD0"),rhs=phi_dD0))

/*
 *  Original SymPy expressions:
 *  "[const double phi_dD0 = invdx0*(-2*phi_i0m1_i1_i2/3 + phi_i0m2_i1_i2/12 + 2*phi_i0p1_i1_i2/3 - phi_i0p2_i1_i2/12),
 *    out_gfs[IDX4(PHID0GF, i0, i1, i2)] = phi_dD0]"
 */
{
   const double phi_i0m2_i1_i2 = in_gfs[IDX4(PHIGF, i0-2,i1,i2)];
   const double phi_i0m1_i1_i2 = in_gfs[IDX4(PHIGF, i0-1,i1,i2)];
   const double phi_i0p1_i1_i2 = in_gfs[IDX4(PHIGF, i0+1,i1,i2)];
   const double phi_i0p2_i1_i2 = in_gfs[IDX4(PHIGF, i0+2,i1,i2)];
   const double phi_dD0 = invdx0*(-(2.0 / 3.0)*phi_i0m1_i1_i2 + ((1.0 / 12.0))*phi_i0m2_i1_i2 + ((2.0 / 3.0))*phi_i0p1_i1_i2 - (1.0 / 12.0)*phi_i0p2_i1_i2);
   out_gfs[IDX4(PHID0GF, i0, i1, i2)] = phi_dD0;
}
